In [31]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
## BADD-- supressing f1 score ill-defined and being set to 0.0 in labels with no true samples warning
import warnings
warnings.filterwarnings("ignore")

In [34]:
from pathlib import Path
import pandas as pd
import numpy as np

In [35]:
import  datatset
import utils
from torch.utils.data import DataLoader

In [36]:
# TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")

# Data

In [ ]:
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [ ]:
def over_sample_class(x):
    if len(x) < 20:
        return x.sample(n=20, replace=True) 
    return x

In [ ]:
def sample_class(x):
    if len(x) > 100:
        return x

In [ ]:
classes_map = utils.load_json(TRAIN_PATH/"classes-map.json")

In [ ]:
counts  = df_train["category_id"].value_counts()

In [ ]:
len(df_train), len(df_train["category_id"].value_counts())

### Over sample and apply data augmentation

In [ ]:
# Over Sample Class
new_df = df_train.groupby('category_id').apply(over_sample_class).reset_index(drop = True)

In [ ]:
# apply data augmentation to labels with less then 21 exampels
labels_aug = counts.keys()[counts < 21] # Get labels that have less then 21 exmaples
augment = new_df["category_id"].isin(labels_aug) # Get list of items to apply aug

In [ ]:
new_df["augment"] = augment #np.zeros(len(new_df)) == 1 # set all augemntaion to zero

In [ ]:
np.sum(new_df.augment)

### Update Labels

In [ ]:
# Update class labels
new_labels = [classes_map[str(i)] for i in new_df["category_id"]]
new_df["new_labels"] = new_labels

In [ ]:
num_calsses = len(counts)

In [ ]:
num_calsses

In [ ]:
from tqdm import tqdm_notebook

# there is a better way to do this
def sample_for_validation(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        if validation_set.get(row["new_labels"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["new_labels"]] = validation_set.get(row["new_labels"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

### Split into train and validation

In [ ]:
new_df["is_valid"] = sample_for_validation(new_df, n=4)

In [ ]:
train_df, valid_df = new_df[~new_df["is_valid"]] , new_df[new_df["is_valid"]]

In [ ]:
# len(train_df), len(valid_df)
# (703500, 14766)

In [ ]:
len(train_df), len(valid_df)

In [ ]:
valid_df.to_csv(TRAIN_PATH/"validation.csv")

In [ ]:
train_df.to_csv(TRAIN_PATH/"train-9.csv")

# RESTART HERE

In [9]:
train_df = pd.read_csv(TRAIN_PATH/"train-9.csv")

In [10]:
num_calsses = len(train_df["category_id"].value_counts())

In [11]:
ds = datatset.HerbDataSet(train_df, TRAIN_PATH, 256, label="new_labels",device="gpu")

In [12]:
data_loader = DataLoader(ds, batch_size=64, shuffle=True)

# Model

In [18]:
import torch
import model
import torchvision.models as models

In [14]:
m = model.get_model(models.resnet50(pretrained=True), p1=0.5, p2=0.5, device="gpu") # no drop out, trying to over fit

In [20]:
model_path = '/home/ubuntu/data-1/nybg2020/train/models-run-9/model-88000.pth'

In [21]:
m.load_state_dict(torch.load(model_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [22]:
import torch.nn as nn
import torch

In [23]:
assert m.fc[-1].out_features == num_calsses

# Tracking

In [24]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

In [25]:
RUN = "run-10"

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/f'logs/{RUN}/train/')

In [26]:
model_path=TRAIN_PATH/f'models-{RUN}/'

In [27]:
if not model_path.exists():
    model_path.mkdir()

In [28]:
import metrics
import train

In [29]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

# Train

In [39]:
import focal_loss

In [41]:
criterion =  focal_loss.FocalLoss()

In [42]:
layer_lrs = [
#     {'params': m.layer1.parameters(), 'lr': 1e-6},
#     {'params': m.layer2.parameters(), 'lr': 1e-6},
#     {'params': m.layer3.parameters(), 'lr': 1e-5},
#     {'params': m.layer4.parameters(), 'lr': 1e-4},
    {'params': m.fc.parameters(), 'lr': 1e-3}
]

In [43]:
optimizer = torch.optim.Adam(layer_lrs, lr=1e-3)

In [44]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 5, m, model_path)

In [45]:
! export CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.run()

Epoch 0/5
